<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a><div align="center">This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.</div>

----

Normally, we would need to initialize a Spark context in order to perform initialization.  However, this is automatically done in this IPython installation, so we skip this part. (Only *one* SparkContext object can be alive at the same time.)

In [1]:
#sc = SparkContext(appName="PythonPi")

Evaluate the following cell to confirm that the PySpark kernel has been started correctly and that a SparkContext is already available in variable `sc`:

In [2]:
sc

<SparkContext master=yarn appName=pyspark-shell>

---

## Exercise 4.A

Let us recall the definition of `wordcount()` from *Exercise 2.D*:

In [3]:
# see: https://docs.python.org/2/library/re.html
import re
punctuation = re.compile(r'[^\w]', re.M)

from operator import add

def wordcount(filename):
    # make a Spark RDD from a text file
    lines1 = sc.textFile(filename)
    # normalize (lowercase, remove punctuation, etc.)
    lines2 = lines1.map(lambda line: punctuation.sub(' ', line).lower())
    # break each line into words (creates a new RDD)
    words1 = lines2.flatMap(lambda line: line.lower().split())
    # final map/reduce step
    words2 = words1.map(lambda word: (word, 1))
    counts = words2.reduceByKey(add)
    return counts

Then apply the conversion to DataFrame from *Exercise 3.B:*

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

def wordcount_df(filename):
    rdd = wordcount(filename)
    df = spark.createDataFrame(rdd, schema=StructType([
        StructField('word', StringType()),
        StructField('nr', IntegerType()),
    ]))
    return df

Finally, load the word counts for both datasets:

In [5]:
wc1 = wordcount_df('hdfs:///shakespeare.txt.gz')

In [6]:
wc2 = wordcount_df('hdfs:///dickens.txt.gz')

To compute the frequencies, we join the two sets and look at the occurrence columns side-by-side.  A problem in this approach is that the `nr` columns have the same name so they would be indistinguishable in the joined DF.

So let us first rename the count columns:

In [7]:
wc1_ = wc1.select(wc1.word, wc1.nr.alias('nr1'))

In [8]:
wc2_ = wc2.select(wc2.word, wc2.nr.alias('nr2'))

Let us compute the joint word count:

In [9]:
joint = wc1_.join(wc2_, on=['word'])

joint.show(5)

+-------------+---+---+
|         word|nr1|nr2|
+-------------+---+---+
|apprehensions|  3|  2|
|    arguments| 11|  2|
|          art|936|  3|
|       biting| 11|  3|
|     briefest|  1|  1|
+-------------+---+---+
only showing top 5 rows



We can now select words that are in Shakespeare's works but not in Dickens':

In [10]:
shakespeare_only = joint.where(joint.nr2 == 0).where(joint.nr1 > 0)

In [11]:
shakespeare_only.take(5)

[]

Ooops! It turns out that `df.join()` does an *inner* join by default, which skips words that are missing on one side.  To capture the information we want, a *full outer join* is needed:

In [12]:
joint = wc1_.join(wc2_, on=['word'], how='full_outer').fillna(0)

In [13]:
dickens_only = joint.where(joint.nr1 == 0).where(joint.nr2 > 0)

In [14]:
dickens_only.take(5)

[Row(word=u'bricked', nr1=0, nr2=1),
 Row(word=u'cautious', nr1=0, nr2=2),
 Row(word=u'codger', nr1=0, nr2=1),
 Row(word=u'coverlid', nr1=0, nr2=1),
 Row(word=u'flashed', nr1=0, nr2=1)]

To take the top used words in Dickens that are not in Shakespeare, use `.orderBy` before taking:

In [19]:
dickens_only.orderBy('nr2', ascending=False).take(5)

[Row(word=u'mr', nr1=0, nr2=1081),
 Row(word=u'bumble', nr1=0, nr2=397),
 Row(word=u'sikes', nr1=0, nr2=354),
 Row(word=u'fagin', nr1=0, nr2=309),
 Row(word=u'brownlow', nr1=0, nr2=173)]

----

Normally the SparkContext is stopped when the computations are done.  We don't do it here in order to be able to run more computations:

In [15]:
#sc.stop()